In [8]:
#importing packages
import pandas as pd
from sklearn.preprocessing import OneHotEncoder


In [7]:
#Part one: Data Exploration
#Importing data
Data = pd.read_csv("https://www.dropbox.com/s/bocjjyo1ehr5auz/insurance_costs_1.csv?dl=1")
Data.head()
#first thing I notice is that Sex, and smoker will need to be dummified 

Data["sex"].value_counts()
#Only male and female recorded, no NA
Data["smoker"].value_counts()
#Only yes and no, no NA
Data.describe()



,age,bmi,charges
count,431.000000,431.000000,431.000000
mean,37.960557,30.768898,12297.098118
std,16.363909,6.111362,11876.527128
min,18.000000,15.960000,1131.506600
25%,22.000000,26.357500,2710.444575
50%,34.000000,30.590000,9866.304850
75%,55.000000,35.272500,14510.872600
max,64.000000,49.060000,55135.402090


In [18]:
#dummifying variables
sex = pd.get_dummies(Data[["sex"]])
Data["male"]= sex["sex_male"]
# enc = OneHotEncoder()
# enc.fit(Data[["sex"]])

# enc.transform(Data[["sex"]]).toarray()